### Trainsients tests with square-wave "pressure" conditioning parameter to visualize attack and decay characteristics

In [ ]:
# Set parameter file and model to load
from utils.myUtils import mostRecent 

# chose an existing pickledParamFile and run loadModelForEvaluation
pickledParamFile = mostRecent("demoModel/*modelParams.pkl")
#pickledParamFile = mostRecent("/home/lonce/ZCODE/PYTORCH/RNNControl/model/*modelParams.pkl")

# choose saved model to run (defaults to the last saved if loadmodelfile is undefined or None)
# Note: looks in same directory as pickledParamFile
loadModelFileName=None
#loadModelFileName='2018-11-23_09-37-53_model_epoch1_step90.pth'

In [ ]:
%run loadModelForEvaluation.ipynb

if 'sample_rate' not in params :   #older versions didn't save sample_rate
    params['sample_rate']=16000

### Primer  
<a id="primer"></a>

In [ ]:
# Make noisy nuetral Priming signal 
myp=np.zeros([1,seqLen,len(params['props'])+1])
# !! PRIMING DOES MATTER - has to be a certain level of noise
myp[0,:,0]=.3*np.random.ranf([seqLen])-.15 #signal
myp[0,:,1]= .45+.1*np.random.ranf([seqLen])     #instrument
myp[0,:,2]= .5*np.random.ranf([seqLen])    #pressure1
#myp[0,:,3]=1-myp[0,:,2]    #pressure2
myp[0,:,3]= np.random.ranf([seqLen])    #pitch
myprimer=torch.tensor(myp, dtype=torch.float)

plt.figure(figsize=(20,3))
plt.plot(np.arange(len(myp[0,:,0])), myp[0,:,0])
plt.grid()
plt.show()

### Ramp up and down in both pitch and volume

In [ ]:
# Generate!
#*************************************
from IPython.display import Audio, display

genlen= int(1*sr) #not including priming

def fsweep(snum) :   #nsnum is sample number not including priming
    t=snum/genlen #snum/params['sample_rate']
    p=2*(.5-np.abs(.5-t))
    v=np.minimum(1.,3.6*(.5-np.abs(.5-t)))
    #if (t<0.01) :
    #    v=0
    #elif t<.3 :
    #    v=.5
    #elif t<.5 :
    #   v=0
    #elif t<.7 :
    #    v=1
    #else :
    #    v=0
    i=0
    # return np.array([i,v,1-v, p])  #pressure2
    return np.array([i,v, p]) 


signal = generate(rnn, genlen, primer=myprimer, paramvect=fsweep)
#plot_signal(signal)

p = [fsweep(t) for t in range(genlen)]
q=[x[1:3] for x in p]

print("signal length is {}".format(len(signal[seqLen:])))
print("p length is {}".format(len(p[1:2])))

plt.figure(figsize=(20,3)) 
plt.plot(np.arange(len(signal[seqLen:])), signal[seqLen:]) #just print one example from the batch
#plt.plot(np.arange(len(signal[256:])), p[:,2]) #just print one example from the batch
plt.plot(np.arange(len(q)), q) #just print one example from the batch
plt.grid()
plt.show()

display(Audio(signal[seqLen:], rate=sr, autoplay=False))

### Three squarewave pressure params of different stengths

In [ ]:
# Generate!
#*************************************
from IPython.display import Audio, display

genlen= int(2*sr) #not including priming

def fsweep(snum) :   #nsnum is sample number not including priming
    t=snum/genlen #snum/params['sample_rate']
    p=2*(.5-np.abs(.5-t))
    #v=.5*2*(.5-np.abs(.5-t))
    if (t<0.02) :
        v=0
    elif t<.222 :
        v=.5
    elif t<.333 :
       v=0
    elif t<.555 :
        v=.1
    elif t<.666 :
        v=0
    elif t<.888 :
        v=1
    else :
        v=0
    i=0
    #return np.array([i,v,1-v, p])  #pressure2
    return np.array([i,v, p]) 


signal = generate(rnn,genlen, primer=myprimer, paramvect=fsweep)
#plot_signal(signal)

p = [fsweep(t) for t in range(genlen)]
q=[x[1:3] for x in p]

print("signal length is {}".format(len(signal[seqLen:])))
print("p length is {}".format(len(p[1:2])))

plt.figure(figsize=(20,3)) 
plt.plot(np.arange(len(signal[seqLen:])), signal[seqLen:]) #just print one example from the batch
#plt.plot(np.arange(len(signal[256:])), p[:,2]) #just print one example from the batch
plt.plot(np.arange(len(q)), q) #just print one example from the batch
plt.grid()
plt.show()

display(Audio(signal, rate=sr, autoplay=False))

In [ ]:
import random
fname = random.choice(datafiles)
fname

### show one of the parameter contours from the original training data

In [ ]:
#bookmark
import random

p=params['props'][1] # amplitude or pressure1
fname = random.choice(datafiles)
print("param {} for fname {}".format(p, fname))

duration=4

from paramManager import paramManager
pm=paramManager.paramManager(params['datadir'], params['paramdir'])
fooparams=pm.getParams(params['datadir'] + '/' + fname) #the path and extension are optional
newtime, newvals = pm.resampleParam(fooparams,p,16000*duration,timestart=0,timeend=duration,verbose=False,overwrite=False)

plt.figure(figsize=(20,1)) 
plt.plot(newtime, newvals) #just print one example from the batch
plt.show()
for i in range (len(fooparams[p]['times'])) :
    print("{},        {}".format( fooparams[p]['times'][i], fooparams[p]['values'][i]))

### Show the transition between primer and parameterized signal

In [ ]:
from IPython.display import Audio

# Generate!
#*************************************
#paramvect = torch.from_numpy(np.array([0,1,0.5])).type(torch.cuda.FloatTensor)
#paramvect = np.array([0,1,0,0]) #pressure2
paramvect = np.array([1,1,.5]) 
signal = generate(rnn,max_length,paramvect=lambda n: paramvect, primer=None)#myprimer)
plt.figure(figsize=(20,3))
plt.plot(np.arange(len(signal[0:512])), signal[0:512])
plt.grid()
plt.show()

display(Audio(data=signal[0:512],rate=sr))


### Interactive square wave pressure parameter contour

In [ ]:
from ipywidgets import interact,interactive
from ipywidgets import FloatSlider

genlen= int(.6*sr) #not including priming

gi=0
gv=0
gp=0


def fsweep(snum) :   #nsnum is sample number not including priming
    
    t=snum/genlen #snum/params['sample_rate']
    if (t<0.1) :
        v=0
    elif t<.8 :
        v=gv
    else :
        v=0
    return np.array([gi,v, gp]) 


def widgetfunc(x,y,z):
    paramvect = np.array([x,y,z])
    global gi
    global gv
    global gp
    
    gi=x
    gv=y
    gp=z
    
    signal = generate(rnn,genlen,primer=myprimer,paramvect = fsweep)
    plt.figure(figsize=(20,3))
    #lot_signal(signal[seqLen:max_length])
    plt.plot(np.arange(len(signal[seqLen:])), signal[seqLen:]) 
    plt.grid()
    plt.show()
    display(Audio(data=signal[0:],rate=sr))
    return signal
    
xslider=FloatSlider(min=0, max=1, step=.01, value=1.0, continuous_update=False, description='instID')
yslider=FloatSlider(min=0, max=1, step=.01, value=1.0, continuous_update=False, description='pressure1')
zslider=FloatSlider(min=0, max=1, step=.01, value=0.5,  continuous_update=False, description='midiPitch')

w = interactive(widgetfunc, x=xslider, y=yslider, z=zslider)
display(w)